In [1]:
import warnings
warnings.filterwarnings('ignore')

from selenium import webdriver
import time
import re
import pandas as pd

In [2]:
url_principal = 'https://www.milanuncios.com'
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--start-maximized')
chrome_options.add_argument('--disable-extensions')

chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--window-size=1280x1696')
chrome_options.add_argument('--user-data-dir=/tmp/user-data')
chrome_options.add_argument('--hide-scrollbars')
chrome_options.add_argument('--enable-logging')
chrome_options.add_argument('--log-level=0')
chrome_options.add_argument('--v=99')
chrome_options.add_argument('--single-process')
chrome_options.add_argument('--data-path=/tmp/data-path')
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--homedir=/tmp')
chrome_options.add_argument('--disk-cache-dir=/tmp/cache-dir')
chrome_options.add_argument('user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36')

#ruta_ = "C:\\Users\\Usuario\\Documents\\chromedriver.exe"
ruta_ = "C:\\Users\\diego.quintero\\Downloads\\chromedriver.exe"

driver = webdriver.Chrome(ruta_, chrome_options=chrome_options)
driver.get(url_principal)
captcha = driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[1]/div/div/div/footer/div/button[2]')
captcha.click()
time.sleep(1)

#search = driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div[1]/div[2]/div[2]/div/div/div[2]/div[1]/div/form/button')
search = driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div[1]/div[2]/div[2]/div/div/div[2]/div[1]/div/form/button')

search.click()
time.sleep(1)

try:
    search2 = driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div[1]/div[2]/div[2]/div/div/div[2]/div[1]/div/form/button/span')
    search2.click()
except:
    pass

try:
    captcha2 = driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[1]/div/div/div/footer/div/button[2]')
    captcha2.click()
except:
    pass


###función para tratar
def getTextAd(WebElement,indexAd):
    try:
        e = WebElement[indexAd].text
    except:
        e = ""
    return e

resultDict= []

num_scrolls = 6 #Normalmente con 6 scroll se pueden recorrer los 30 items que tiene cada página.
num_items = 40 #Normalmente milanuncios tiene 200 items en el scroll. Se ponen menos 

advice = dict()

for i in range(num_items):
    for scroll in range(num_scrolls):
        #driver.execute_script("window.scrollBy(0,3000)", "")
        driver.execute_script("window.scrollBy(0,3000)", "")
        time.sleep(1)
        
    principals_links = driver.find_elements_by_xpath(
        '//*[@id="app"]/div[2]/div[3]/div[1]/div[2]/div[2]/div[1]/main/div[2]/article['+str(i)+']/div/div[1]/div/h2/a')
    secundary_links = driver.find_elements_by_xpath(
        '//*[@id="app"]/div[2]/div[3]/div[1]/div[2]/div[2]/div[1]/main/div[2]/article['+str(i)+']/div/div[2]/div[1]/div[1]/div[1]/a')
    descriptions = driver.find_elements_by_xpath(
        '//*[@id="app"]/div[2]/div[3]/div[1]/div[2]/div[2]/div[1]/main/div[2]/article['+str(i)+']/div/div[2]/div[1]/div[1]/div[2]/p')
    price = driver.find_elements_by_xpath(
        '//*[@id="app"]/div[2]/div[3]/div[1]/div[2]/div[3]/div[1]/main/div[2]/article['+str(i)+']/div/div[2]/div[2]/div/div/p')
    location_link = driver.find_elements_by_xpath( 
        '//*[@id="app"]/div[2]/div[3]/div[1]/div[2]/div[2]/div[1]/main/div[2]/article['+str(i)+']/div/div[2]/div[1]/div[1]/div[1]/p/span')        

    type_car = driver.find_elements_by_xpath( 
        '//*[@id="app"]/div[2]/div[3]/div[1]/div[2]/div[2]/div[1]/main/div[2]/article['+str(i)+']/div/div[2]/div[2]/div/ul/li/span/span')
    
    #patron oferta/demanda
    demand_of = driver.find_elements_by_xpath( 
        '//*[@id="app"]/div[2]/div[3]/div[1]/div[2]/div[2]/div[1]/main/div[2]/article['+str(i)+']/header/p')
    
    #Patron año del carro
    year_vehicle = driver.find_elements_by_xpath(
       '//*[@id="app"]/div[2]/div[3]/div[1]/div[2]/div[2]/div[1]/main/div[2]/article['+str(i)+']/div/div[2]/div[2]/div/ul/li[1]/span/span')
    
    #Color del vehículo
    color_vehicle = driver.find_elements_by_xpath(
       '//*[@id="app"]/div[2]/div[3]/div[1]/div[2]/div[2]/div[1]/main/div[2]/article['+str(i)+']/div/div[2]/div[2]/div/ul/li[3]/span/span')
        
    #Manual o automático:    
    manual_aut = driver.find_elements_by_xpath(
       '//*[@id="app"]/div[2]/div[3]/div[1]/div[2]/div[2]/div[1]/main/div[2]/article['+str(i)+']/div/div[2]/div[2]/div/ul/li[3]')

    #Número de puertas
    num_gates = driver.find_elements_by_xpath(
      '//*[@id="app"]/div[2]/div[3]/div[1]/div[2]/div[2]/div[1]/main/div[2]/article['+str(i)+']/div/div[2]/div[2]/div/ul/li[4]')

    #CV   
    cv = driver.find_elements_by_xpath(
                '//*[@id="app"]/div[2]/div[3]/div[1]/div[2]/div[2]/div[1]/main/div[2]/article[30]/div/div[2]/div[2]/div/ul/li[5]/span/span')
    
    print("i-->", i," scroll--> ", scroll)
    
    advice = dict()
    
    for ind, link in enumerate(principals_links):
        principal = link.get_attribute('href')
        subcategory = secundary_links[ind].get_attribute('href')
        advice['ID'] = re.sub('-', ' ', re.findall('(\d+)(?=\s*.htm)', principal)[0])
        advice['Nombre'] = re.sub('-', ' ', re.findall('\.com/.*[a-z]/(.*)-[0-9].*\.htm', principal)[0])
        advice['Descripcion'] = descriptions[ind].text
        advice['Categoria'] = re.sub('-', ' ', re.findall('com/(.*)/', principal)[0])
        advice['Subcategoria'] = re.sub('-', ' ', re.findall('com/(.*)/', subcategory)[0])        
        advice['Link'] = principal
        advice['locacion'] = location_link[ind].text
        advice['Oferta_Demanda'] = demand_of[ind].text 
        
        print("ind--antes de campos no permanentes>", ind)
        
        advice['Num_gates']= num_gates[ind].text if num_gates else ""
        advice['Precio']= price[ind].text if price else ""
        advice['Color']= color_vehicle[ind].text if color_vehicle else ""
        advice['Manual_Automatico']= manual_aut[ind].text if manual_aut else ""
        advice['Year']= year_vehicle[ind].text if year_vehicle  else ""
        advice['Tipo']= type_car[ind].text if type_car else ""
        advice['Other_skills']= cv[ind].text if cv else ""
       
        resultDict.append(advice)
                
        print("Ind-después de campos no permamentes->", ind)
    try:
        paginator = driver.find_element_by_xpath('//nav/button[2]')
        print("If paginator work i-->", i," scroll--> ", scroll)
        paginator.click()
        
    except:
        time.sleep(2)
        paginator = driver.find_element_by_xpath('//nav/button[2]')
        print("If paginator do not work i-->", i," scroll--> ", scroll)
        paginator.click()
    time.sleep(1)

driver.close()

i--> 0  scroll-->  5
If paginator work i--> 0  scroll-->  5
i--> 1  scroll-->  5
ind--antes de campos no permanentes> 0
Ind-después de campos no permamentes-> 0
If paginator work i--> 1  scroll-->  5
i--> 2  scroll-->  5
ind--antes de campos no permanentes> 0
Ind-después de campos no permamentes-> 0
If paginator work i--> 2  scroll-->  5
i--> 3  scroll-->  5
ind--antes de campos no permanentes> 0
Ind-después de campos no permamentes-> 0
If paginator work i--> 3  scroll-->  5
i--> 4  scroll-->  5
ind--antes de campos no permanentes> 0
Ind-después de campos no permamentes-> 0
If paginator work i--> 4  scroll-->  5
i--> 5  scroll-->  5
ind--antes de campos no permanentes> 0
Ind-después de campos no permamentes-> 0
If paginator work i--> 5  scroll-->  5
i--> 6  scroll-->  5
ind--antes de campos no permanentes> 0
Ind-después de campos no permamentes-> 0
If paginator work i--> 6  scroll-->  5
i--> 7  scroll-->  5
ind--antes de campos no permanentes> 0
Ind-después de campos no permamentes-> 0

In [3]:
coche_list = pd.DataFrame(resultDict)

In [4]:
coche_list.head()

,ID,Nombre,Descripcion,Categoria,Subcategoria,Link,locacion,Oferta_Demanda,Num_gates,Precio,Color,Manual_Automatico,Year,Tipo,Other_skills
0,407629306,sofa,"sofa de calidad. blanco lavable todo el sofa, ...",sillas sofas y sillones,sillas sofas y sillones en olocau valencia,https://www.milanuncios.com/sillas-sofas-y-sil...,Olocau,OFERTA,,,,,,,
1,427222643,pago mas que todos coches financiados,Compramos coches forgonetas todoterrenos 4x4 a...,coches de segunda mano,coches de segunda mano en alicante,https://www.milanuncios.com/coches-de-segunda-...,Alicante/Alacant,DEMANDA,,,,,Automático,Automático,
2,416751617,tienes algun coche con embargo o reserva,"con embargo, reserva, precinto, leasing, para ...",coches de segunda mano,coches de segunda mano en zaragoza,https://www.milanuncios.com/coches-de-segunda-...,Zaragoza,DEMANDA,,,,,Automático,Automático,
3,429709637,compramos tu coche compramos tu coche,vehículos con deudas EMBARGADOS Reserva domini...,coches de segunda mano,coches de segunda mano en salamanca,https://www.milanuncios.com/coches-de-segunda-...,Salamanca,DEMANDA,,,,,Automático,Automático,
4,429975172,cuadro de equipo del real madrid,Cuadro de equipo de fútbol Del Real Madrid por...,pinturas,pinturas en palma de mallorca baleares,https://www.milanuncios.com/pinturas/cuadro-de...,Palma de Mallorca,OFERTA,,,,,,,


In [5]:
#coche_list.to_csv(r'C:/Users/Usuario/.spyder-py3/coches_precios.csv', index=None, header=True, encoding='utf-8-sig')
coche_list.to_csv(r'C:/Users/diego.quintero/Downloads/Informacion_anuncios_Adrian.csv', index=None, header=True, encoding='utf-8-sig')